In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from env import get_connection

from acquire import get_curriculum_logs
from prepare import prep_curriculum

pd.set_option('display.max_columns', None)

import warnings
warnings.filterwarnings('ignore')

In [2]:
df = get_curriculum_logs()
df = prep_curriculum(df)

In [3]:
df.head()

,date,time,path,user_id,cohort_id,ip,id,name,start_date,end_date,program_id,count,ip_count
0,2018-01-26,09:56:02,java-ii,1,8.0,97.105.19.61,8,Hampton,2015-09-22,2016-02-06,1,12177.0,57655
1,2018-01-26,09:56:05,java-ii/object-oriented-programming,1,8.0,97.105.19.61,8,Hampton,2015-09-22,2016-02-06,1,6014.0,57655
2,2018-01-26,09:56:06,slides/object_oriented_programming,1,8.0,97.105.19.61,8,Hampton,2015-09-22,2016-02-06,1,483.0,57655
3,2018-01-26,09:56:24,javascript-i/conditionals,2,22.0,97.105.19.61,22,Teddy,2018-01-08,2018-05-17,2,6330.0,57655
4,2018-01-26,09:56:41,javascript-i/loops,2,22.0,97.105.19.61,22,Teddy,2018-01-08,2018-05-17,2,6455.0,57655


##  Is there a cohort that referred to a lesson significantly more than other cohorts seemed to gloss over?

In [4]:
# removing values with only the slash
df = df[df['path']!='/']

In [5]:
# counting the number of times each path value appears
df['count'] = df.groupby('path')['path'].transform('count')

In [6]:
# splitting data into 3 data frames, 1 for each program
cohort_path_1 = pd.DataFrame(df['path'].drop_duplicates())
cohort_path_2 = pd.DataFrame(df['path'].drop_duplicates())
cohort_path_3 = pd.DataFrame(df['path'].drop_duplicates())

In [7]:
# pulling cohort names for each program 
prgm_1 = df[df['program_id']==1]['name'].unique()
prgm_2 = df[df['program_id']==2]['name'].unique()
prgm_3 = df[df['program_id']==3]['name'].unique()

In [8]:
# filling data frames with information for each path
for name in prgm_1:
    cohort_path_1 = pd.merge(cohort_path_1, pd.DataFrame({f'{name}':df[df['name']==f'{name}'].value_counts('path')}), 
         how='left',on='path')
    
    cohort_path_1[f'{name}'] = cohort_path_1[f'{name}'].fillna(0)

for name in prgm_2:
    cohort_path_2 = pd.merge(cohort_path_2, pd.DataFrame({f'{name}':df[df['name']==f'{name}'].value_counts('path')}), 
         how='left',on='path')
    
    cohort_path_2[f'{name}'] = cohort_path_2[f'{name}'].fillna(0)
    
for name in prgm_3:
    cohort_path_3 = pd.merge(cohort_path_3, pd.DataFrame({f'{name}':df[df['name']==f'{name}'].value_counts('path')}), 
         how='left',on='path')
    
    cohort_path_3[f'{name}'] = cohort_path_3[f'{name}'].fillna(0)

In [9]:
# creating new column with the difference of the most times a lesson was accessed vs the least
cohort_path_1['diff'] = cohort_path_1.max(axis=1) - cohort_path_1.min(axis=1)
cohort_path_2['diff'] = cohort_path_2.max(axis=1) - cohort_path_2.min(axis=1)
cohort_path_3['diff'] = cohort_path_3.max(axis=1) - cohort_path_3.min(axis=1)

In [10]:
# determing highest differences in number of times a lessone was accessed
cohort_path_1[cohort_path_1['diff']>200]

,path,Hampton,Arches,Quincy,Kings,Lassen,Glacier,Denali,Joshua,Olympic,Badlands,Ike,Franklin,diff
8,javascript-i,36.0,294.0,6.0,17.0,233.0,11.0,0.0,1.0,128.0,1.0,7.0,2.0,294.0
15,spring,43.0,192.0,8.0,3.0,222.0,2.0,0.0,0.0,24.0,0.0,5.0,2.0,222.0
20,index.html,1.0,1.0,6.0,84.0,877.0,0.0,0.0,8.0,28.0,0.0,6.0,0.0,877.0
34,html-css,21.0,215.0,12.0,10.0,174.0,21.0,0.0,5.0,69.0,2.0,12.0,1.0,215.0
84,javascript-ii,31.0,204.0,7.0,10.0,107.0,6.0,0.0,0.0,58.0,0.0,4.0,2.0,204.0
131,java-iii,57.0,153.0,3.0,8.0,224.0,0.0,0.0,1.0,25.0,0.0,5.0,3.0,224.0


**For program id 1 the cohorts that accessed lessons more than other cohorts was the Lassen cohort**

In [11]:
cohort_path_2[cohort_path_2['diff']>1200]

,path,Teddy,Sequoia,Niagara,Pinnacles,Mammoth,Ulysses,Voyageurs,Wrangell,Xanadu,Yosemite,Staff,Zion,Andromeda,Betelgeuse,Ceres,Deimos,Europa,Fortuna,Apex,Ganymede,Hyperion,Bash,Jupiter,Kalypso,Luna,Marco,Neptune,Oberon,diff
0,java-ii,509.0,129.0,17.0,50.0,10.0,425.0,756.0,325.0,502.0,274.0,1260.0,624.0,398.0,481.0,681.0,511.0,375.0,657.0,666.0,641.0,688.0,268.0,755.0,417.0,255.0,40.0,5.0,0.0,1260.0
8,javascript-i,615.0,202.0,23.0,69.0,6.0,526.0,884.0,560.0,722.0,452.0,1817.0,897.0,516.0,717.0,1003.0,896.0,506.0,785.0,669.0,760.0,910.0,374.0,926.0,631.0,676.0,907.0,272.0,136.0,1811.0
15,spring,744.0,364.0,65.0,56.0,12.0,409.0,650.0,293.0,454.0,381.0,1403.0,672.0,487.0,299.0,562.0,533.0,324.0,636.0,600.0,508.0,526.0,311.0,621.0,339.0,74.0,43.0,8.0,2.0,1401.0
16,appendix,505.0,170.0,20.0,25.0,9.0,176.0,394.0,209.0,325.0,144.0,1288.0,296.0,161.0,188.0,411.0,343.0,243.0,356.0,450.0,561.0,400.0,174.0,421.0,180.0,92.0,82.0,15.0,10.0,1279.0
34,html-css,241.0,168.0,6.0,56.0,3.0,562.0,528.0,411.0,597.0,384.0,1284.0,675.0,375.0,457.0,766.0,773.0,353.0,585.0,738.0,504.0,439.0,265.0,784.0,495.0,307.0,479.0,313.0,21.0,1281.0
131,java-iii,732.0,105.0,29.0,45.0,13.0,412.0,770.0,337.0,497.0,351.0,1393.0,753.0,433.0,472.0,682.0,523.0,451.0,786.0,663.0,651.0,701.0,324.0,795.0,456.0,259.0,40.0,9.0,1.0,1392.0
610,toc,83.0,36.0,8.0,12.0,1.0,89.0,272.0,991.0,443.0,697.0,911.0,1465.0,638.0,610.0,911.0,750.0,964.0,1293.0,810.0,822.0,1022.0,404.0,1866.0,495.0,373.0,910.0,451.0,101.0,1865.0
617,search/search_index.json,103.0,45.0,7.0,9.0,1.0,142.0,328.0,504.0,577.0,361.0,1349.0,700.0,318.0,761.0,1380.0,746.0,742.0,1020.0,1497.0,1095.0,670.0,660.0,998.0,467.0,405.0,241.0,77.0,9.0,1496.0


**For program id 2 the cohorts that accessed lessons more than other cohorts it was Voyageurs, Zion, Fortuna, Apex, Hyperion, and Jupiter**

In [12]:
cohort_path_3[cohort_path_3['diff']>600]

,path,Bayes,Curie,Darden,Easley,Florence,diff
1391,1-fundamentals/modern-data-scientist.jpg,650.0,467.0,470.0,7.0,61.0,643.0
1406,1-fundamentals/1.1-intro-to-data-science,640.0,461.0,460.0,8.0,64.0,632.0
1407,1-fundamentals/AI-ML-DL-timeline.jpg,648.0,465.0,470.0,7.0,61.0,641.0
1766,fundamentals/modern-data-scientist.jpg,24.0,100.0,308.0,379.0,627.0,603.0
1781,sql/mysql-overview,27.0,125.0,774.0,295.0,203.0,747.0
1795,classification/overview,35.0,136.0,1109.0,445.0,60.0,1074.0
2003,classification/scale_features_or_not.svg,28.0,96.0,943.0,463.0,60.0,915.0


**For program id 3 the cohort that accessed the lessons more than other cohorts it was Darden**

##  Is there any suspicious activity, such as users/machines/etc accessing the curriculum who shouldn’t be? Does it appear that any web-scraping is happening? Are there any suspicious IP addresses?

In [13]:
# determing how many ip addresses show up less than 5 times and listing them
sus_ip = pd.DataFrame(df[df['ip_count']<5]['ip'].drop_duplicates())['ip']
sus_ip

426        107.77.100.125
670          99.203.26.41
909       209.205.122.186
1465      184.203.148.135
1965       173.173.116.10
               ...       
800718       70.117.2.210
800747       172.58.97.72
801257       134.250.1.82
801298      174.197.2.198
801378      107.77.169.13
Name: ip, Length: 2042, dtype: object

**There are 236 ip addresses that were used less than 5 times and should all be considered suspisions id addresses**

## What topics are grads continuing to reference after graduation and into their jobs (for each program)?

In [14]:
# determing which lessons are accessed after a student graduates by program id
cohort_grad_1 = pd.DataFrame({'count':df[(df['program_id']==1) & (df['date']>df['end_date'])].value_counts('path')})

cohort_grad_2 = pd.DataFrame({'count':df[(df['program_id']==2) & (df['date']>df['end_date'])].value_counts('path')})

cohort_grad_3 = pd.DataFrame({'count':df[(df['program_id']==3) & (df['date']>df['end_date'])].value_counts('path')})

In [15]:
# only showing top results
cohort_grad_1[cohort_grad_1['count']>500]

,count
path,
index.html,1011
javascript-i,736
html-css,542
spring,501


**For program id 1 the lessons accessed the most after graduation was index.html and javascript-i**

In [16]:
cohort_grad_2[cohort_grad_2['count']>3000]

,count
path,
javascript-i,4229
spring,3760
search/search_index.json,3562
html-css,3136
java-iii,3058


**For program id 2 the lessons accessed the most after graduation was javascript-i and spring**

In [17]:
cohort_grad_3[cohort_grad_3['count']>200]

,count
path,
search/search_index.json,493
sql/mysql-overview,275
classification/overview,266
classification/scale_features_or_not.svg,219


**For program id 3 the lessons accessed the most after graduation was mysql_overview and classification overview**

## Which lessons are least accessed?

In [18]:
# showing which lessons where accessed the least
df[(df['count']==1) & (df['path'].str.contains('/')) & (df['path'].str.count('\.')==0)
  & (df['name']!= 'Staff') & (df['ip_count']>5)]

,date,time,path,user_id,cohort_id,ip,id,name,start_date,end_date,program_id,count,ip_count
203,2018-01-26,13:39:02,javascript/google-maps-api,38,21.0,97.105.19.61,21,Sequoia,2017-09-27,2018-02-15,2,1.0,57655
519,2018-01-27,07:49:25,teams/13,1,8.0,72.177.226.58,8,Hampton,2015-09-22,2016-02-06,1,1.0,201
2623,2018-01-31,15:22:06,javascript/loops,47,21.0,97.105.19.61,21,Sequoia,2017-09-27,2018-02-15,2,1.0,57655
15743,2018-03-01,21:07:05,prework/cli/07-editing-files-with-vim,114,8.0,173.173.119.75,8,Hampton,2015-09-22,2016-02-06,1,1.0,17
20370,2018-03-11,14:39:16,java-ii/file-op,1,8.0,72.177.226.58,8,Hampton,2015-09-22,2016-02-06,1,1.0,201
...,...,...,...,...,...,...,...,...,...,...,...,...,...
748553,2021-03-11,09:09:49,java-iii/sessions-and-cookies/style/css,790,132.0,99.173.134.103,132,Kalypso,2020-11-02,2021-05-04,2,1.0,1104
749856,2021-03-12,10:00:24,further-reading/spring,20,22.0,104.190.242.242,22,Teddy,2018-01-08,2018-05-17,2,1.0,1477
755399,2021-03-17,10:44:31,javascript-i/intruduction/operators,923,138.0,173.174.231.66,138,Neptune,2021-03-15,2021-09-03,2,1.0,151
764977,2021-03-23,14:08:52,sql/database,949,137.0,136.49.16.164,137,Florence,2021-03-15,2021-09-03,3,1.0,209


**There are 134 lessons that were accessed only once that are all equal in least accessed**